In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

print(train_data['text'])

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# 停用詞
stop_words = set(stopwords.words('english'))

def replace_some(Str):
  Str = re.sub(r'http[s]?://\S+', "", Str)
  Str = re.sub(r'@\S+', "", Str)
  Str = re.sub(r"[^a-zA-Z\s]","",Str.lower())
  Str = Str.split()
  Str = [word for word in Str if word not in stop_words]
  return ' '.join(Str) #隔著' '黏合在一起

train_data['text'] = train_data['text'].apply(replace_some)
test_data['text'] = test_data['text'].apply(replace_some)

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['text'])  #建立詞典
sequences = tokenizer.texts_to_sequences(train_data['text'])
#依照詞典index將文本轉成一連串數字

x = pad_sequences(sequences)  #把每個text的sequences長度變得一樣(增長)''',maxlen = len(tokenizer.word_index)+1,padding='post'''
y = np.array(train_data['target'])

In [4]:
model = Sequential([
    Embedding(input_dim = len(tokenizer.word_index)+1,output_dim=50),  # 嵌入層
    SimpleRNN(32, activation='tanh'),  # RNN 層
    Dense(1, activation='sigmoid')  # 輸出層，二元分類
])
#嵌入層的input_dim是輸入資料是幾維／一單位(+1是因為詞典還包含所有詞的索引?)
#tanh則是把輸出控制在1~-1
#總之50跟32是chatgpt的建議(?)他說這兩個數字適合
#輸出層是1是因為最後只輸出target，sigmoid是給二元分類的(0~1)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#optimizer:優化器
#loss:損失函數
#metrics:評估指標。選準確率(預測對的筆數／所有筆數)

model.fit(x, y, epochs=5, batch_size=32, validation_split=0.2)
#epochs:訓練5趟
#batch_size:一次取多少數據進行訓練
#validation:多少數據用來驗證

Epoch 1/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.6220 - loss: 0.6528 - val_accuracy: 0.7571 - val_loss: 0.5048
Epoch 2/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.9011 - loss: 0.2713 - val_accuracy: 0.7255 - val_loss: 0.5632
Epoch 3/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9691 - loss: 0.1088 - val_accuracy: 0.7380 - val_loss: 0.5885
Epoch 4/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9754 - loss: 0.0796 - val_accuracy: 0.7538 - val_loss: 0.6214
Epoch 5/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.9747 - loss: 0.0669 - val_accuracy: 0.7584 - val_loss: 0.6521


In [7]:
sequences_test = tokenizer.texts_to_sequences(test_data['text'])
x_test = pad_sequences(sequences_test)
y_test = (model.predict(x_test)>0.7).astype("int32")
#以0.7為界是目前結果最佳的

102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [10]:
submission = pd.DataFrame({
    'id':test_data['id'],
    'target':y_test.flatten()
})
#flatten:轉換成1D陣列
submission.to_csv('lala_submission.csv',index = False)

In [15]:
print("kaggle accuracy:0.75390")
print("\nreport:")

report = pd.DataFrame({
    'accuracy':[0.9747],'loss':[0.0669],'val_acurracy':[0.7584],'val_loss':[0.6521]
})
report.index = ["Epoch 5/5"]
print(report)

kaggle accuracy:0.75390

report:
           accuracy    loss  val_acurracy  val_loss
Epoch 5/5    0.9747  0.0669        0.7584    0.6521
